# Radial basis functions

## Linear combination of general basis functions

This notebook contains studies on radial basis functions interpolation, to be used as part of surrogate optimization. This optimization method uses a *surrogate* (also known as metamodel, or surface response model) model to appoximate a given function $f:R^d\rightarrow R$.

Follows the basic idea of approximating the function as a linear conbination of basis functions of $\vec{x}$:

\begin{equation}
\mathbf{h} = w_0\mathbf{\varphi_0}(\mathbf{x}) + w_1\mathbf{\varphi_1}(\mathbf{x})+\dots+w_n\mathbf{\varphi_n}(\mathbf{x})=\sum_{i=1}^n w_i\mathbf{\varphi_i}(\mathbf{x})
\end{equation}

Written in matrix notation:

\begin{equation}
    \mathbf{h} =
    \left[
    {\begin{array}{cc}
    \mathbf{\varphi_0}(\mathbf{x}) & \mathbf{\varphi_1}(\mathbf{x}) & \dots & \mathbf{\varphi_n}(\mathbf{x})
    \end{array} }
    \right]\mathbf{w}
\end{equation}

From this general form, derives linear, polynomial and other basis functions combinations.

>Note that $\mathbf{\varphi_i}$ arefunctions of the **vector** $\mathbf{x}$. For linear combination, normally each function $\mathbf{\varphi_i}$ is dependent of a single component of $\mathbf{x}$, but in a general form it depends of all the componentes of $\mathbf{x}$, as does radial basis functions.

## Radial basis functions

Radial basis functions interpolation uses a special set of functions, based on univariate (radial) functions and a set of observed data.

Given a set of $m$ points in $R^d$, let's refer to them as $\left\{\mathbf{\hat{x}}_1,\ldots, \mathbf{\hat{x}}_m \right\}$, where the function $f:R^d \rightarrow R$ evaluation is known. The function is approximated by the linear combination of $m$ basis functions $\mathbf{\varphi_i(x)}$:

\begin{equation}
    \mathbf{h} =
    \left[
    {\begin{array}{cc}
    \mathbf{\varphi_0}(\mathbf{x}) & \mathbf{\varphi_1}(\mathbf{x}) & \dots & \mathbf{\varphi_m}(\mathbf{x})
    \end{array} }
    \right]\mathbf{w}
\end{equation}

This functions are defined as:

$$\varphi_i\left(\mathbf{x}\right)=\gamma( \Vert \mathbf{x}-\mathbf{\hat{x}_i} \Vert)$$

Where $\Vert \mathbf{v} \Vert$, denotes the norm of the vector $\mathbf{v}$ and $\gamma$ is an univariate, normally continuous function called a **radial basis function**.